# FUA simplification with [`cityseer`](https://github.com/benchmark-urbanism/cityseer-api)

* **Simons, G.** (2023). *The cityseer Python package for pedestrian-scale network-based urban analysis*. Environment and Planning B: Urban Analytics and City Science, 50(5), 1328-1344. https://doi.org/10.1177/23998083221133827

In [1]:
%load_ext watermark
%watermark

Last updated: 2024-12-05T11:45:54.033468+01:00

Python implementation: CPython
Python version       : 3.12.7
IPython version      : 8.30.0

Compiler    : Clang 17.0.6 
OS          : Darwin
Release     : 24.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



In [2]:
import pathlib
import time

import cityseer
import geopandas

from core import utils

%watermark -w
%watermark -iv

Watermark: 2.5.0

cityseer : 4.17.2
core     : 0.1.dev170+gc3e76d8.d20241205
geopandas: 1.0.1



In [3]:
def run_cityseer(city: str) -> geopandas.GeoDataFrame:
    """Run the ``cityseer`` workflow as laid out in
    [https://benchmark-urbanism.github.io/cityseer-examples/examples/graph_cleaning.html#manual-cleaning]
    with ***all default arguments***.

    Parameters
    ----------
    city : str
        City anme. See ``utils.city_fua.keys()``.

    Returns
    -------
    geopandas.GeoDataFrame
        Resultant simplified network edges – only geometry column.
        Will be written to ``.parquet``.
    """
    edges = utils.read_original(city)
    graph = cityseer.tools.io.nx_from_generic_geopandas(edges)
    graph = cityseer.tools.graphs.nx_remove_filler_nodes(graph)
    graph = cityseer.tools.graphs.nx_remove_dangling_nodes(graph)
    graph = cityseer.tools.graphs.nx_split_opposing_geoms(graph)
    graph = cityseer.tools.graphs.nx_remove_filler_nodes(graph)
    graph = cityseer.tools.graphs.nx_consolidate_nodes(graph)
    graph = cityseer.tools.graphs.nx_remove_filler_nodes(graph)
    graph = cityseer.tools.graphs.nx_iron_edges(graph)
    return cityseer.tools.io.geopandas_from_nx(graph, crs=edges.crs).rename_geometry(
        "geometry"
    )[["geometry"]]

In [5]:
output_base = pathlib.Path("..", "..", "data")

In [6]:
global_start = time.time()

for city, fua in utils.city_fua.items():
    print("===================================================================")
    print(f"Starting {city}")
    print("-------------------------")
    print("\n\n")

    fua_start = time.time()
    simplified = run_cityseer(city)
    output_fua = output_base / pathlib.Path(f"{fua}", "cityseer", f"{fua}.parquet")
    simplified.to_parquet(output_fua)
    fua_end = time.time()
    fua_duration = round((fua_end - fua_start) / 60.0, 3)

    print(f"\t{city} completed in: {fua_duration} minutes")
    print("\n\n\n\n")

print("~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ")
print(" ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~")
print("~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ")
print(" ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~")

global_end = time.time()
global_duration = round((global_end - global_start) / 60.0, 3)

print(f"Total runtime: {global_duration} minutes")

Starting Aleppo
-------------------------





100%|██████████| 78908/78908 [00:05<00:00, 15605.52it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.
100%|██████████| 78908/78908 [00:00<00:00, 243062.34it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 64445/64445 [00:02<00:00, 26347.83it/s]
INFO:cityseer.tools.graphs:Removing dangling nodes.
100%|██████████| 28776/28776 [00:00<00:00, 719327.80it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 28710/28710 [00:00<00:00, 1014203.99it/s]
INFO:cityseer.tools.util:Creating edges STR tree.
100%|██████████| 43034/43034 [00:00<00:00, 1415257.36it/s]
INFO:cityseer.tools.graphs:Splitting opposing edges.
100%|██████████| 28571/28571 [00:01<00:00, 19287.70it/s]
INFO:cityseer.tools.graphs:Squashing opposing nodes
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 25.
100%|██████████| 43576/43576 [00:00<00:00, 109593.99it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 28571/28571 [00:00<00:

	Aleppo completed in: 0.344 minutes





Starting Auckland
-------------------------





100%|██████████| 60364/60364 [00:04<00:00, 14590.03it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.
100%|██████████| 60364/60364 [00:00<00:00, 190614.01it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 57031/57031 [00:02<00:00, 19073.53it/s]
INFO:cityseer.tools.graphs:Removing dangling nodes.
100%|██████████| 9123/9123 [00:00<00:00, 486042.09it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 8980/8980 [00:00<00:00, 475104.38it/s]
INFO:cityseer.tools.util:Creating edges STR tree.
100%|██████████| 12214/12214 [00:00<00:00, 1203015.90it/s]
INFO:cityseer.tools.graphs:Splitting opposing edges.
100%|██████████| 8898/8898 [00:00<00:00, 17139.89it/s]
INFO:cityseer.tools.graphs:Squashing opposing nodes
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 25.
100%|██████████| 12573/12573 [00:00<00:00, 99041.20it/s] 
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 8898/8898 [00:00<00:00, 46836

	Auckland completed in: 0.2 minutes





Starting Bucaramanga
-------------------------





...

	Liège completed in: 0.25 minutes





Starting Salt Lake City
-------------------------





100%|██████████| 50917/50917 [00:03<00:00, 14873.26it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.
100%|██████████| 50917/50917 [00:00<00:00, 197498.98it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 46988/46988 [00:02<00:00, 19518.39it/s]
INFO:cityseer.tools.graphs:Removing dangling nodes.
100%|██████████| 9539/9539 [00:00<00:00, 498877.37it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 9473/9473 [00:00<00:00, 715490.92it/s]
INFO:cityseer.tools.util:Creating edges STR tree.
100%|██████████| 13358/13358 [00:00<00:00, 1214030.61it/s]
INFO:cityseer.tools.graphs:Splitting opposing edges.
100%|██████████| 9431/9431 [00:00<00:00, 23919.74it/s]
INFO:cityseer.tools.graphs:Squashing opposing nodes
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 25.
100%|██████████| 13453/13453 [00:00<00:00, 208025.08it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 9431/9431 [00:00<00:00, 75407

	Salt Lake City completed in: 0.158 minutes





Starting Wuhan
-------------------------





100%|██████████| 92667/92667 [00:05<00:00, 15955.65it/s]
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 1.
100%|██████████| 92667/92667 [00:00<00:00, 255404.18it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 82891/82891 [00:04<00:00, 20387.45it/s]
INFO:cityseer.tools.graphs:Removing dangling nodes.
100%|██████████| 19549/19549 [00:00<00:00, 648459.40it/s]
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 19433/19433 [00:00<00:00, 806953.08it/s]
INFO:cityseer.tools.util:Creating edges STR tree.
100%|██████████| 29120/29120 [00:00<00:00, 1223976.15it/s]
INFO:cityseer.tools.graphs:Splitting opposing edges.
100%|██████████| 19369/19369 [00:01<00:00, 10544.77it/s]
INFO:cityseer.tools.graphs:Squashing opposing nodes
INFO:cityseer.tools.graphs:Merging parallel edges within buffer of 25.
100%|██████████| 31499/31499 [00:00<00:00, 72791.23it/s] 
INFO:cityseer.tools.graphs:Removing filler nodes.
100%|██████████| 19369/19369 [00:00<00:0

	Wuhan completed in: 0.347 minutes





~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ 
 ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
Total runtime: 1.823 minutes
